In [1]:
import numpy as np

Some Helper Methods

In [2]:
def calculate_minimal_singular_value(R, k, lambda_i):
    # Extract submatrices from R for singular value decomposition
    A = R[:k, :-k]
    B = R[:k, :k]
    C = R[:-k, :-k]

    concatenated_R = np.concatenate((A - lambda_i * B, C), axis=0)
    _, singular_values, right_singular_vectors = np.linalg.svd(A - lambda_i * B, full_matrices=False)

    # Find the index of the minimal singular value
    min_singular_value_index = np.argmin(singular_values)

    # Extract the minimal singular value and the corresponding right singular vector
    sigma_lambda_i = singular_values[min_singular_value_index]
    w_lambda_i = right_singular_vectors[min_singular_value_index]

    return sigma_lambda_i, w_lambda_i

Implementation of the Schmid_DMD_method

In [3]:
def schmid_DMD_method(Xm, Ym):
    # Step 2: Thin SVD
    U, Sigma, Vh = np.linalg.svd(Xm, full_matrices=False)

    # Step 3: Determine numerical rank k
    k = np.sum(Sigma > np.finfo(float).eps * max(Xm.shape))

    # Step 4: Truncate SVD
    Uk = U[:, :k]
    Vk = Vh[:, :k]
    Sigma_k = np.diag(Sigma[:k])

    # Step 5: Compute Schmid's Rayleigh Quotient
    Sk = Uk.conj().T @ Ym @ Vk @ np.linalg.inv(Sigma_k)

    # Step 6: Compute Eigenpairs of Sk
    Lambda_k, Wk = np.linalg.eig(Sk)

    # Step 7: Output Ritz vectors Zk and eigenvalues Λk
    Zk = Uk @ Wk

    return Zk, np.diag(Lambda_k)


Example Usage of the Method:

In [4]:
# Test method:
# Replace Xm and Ym with your data
m_size = 5
n_size = 3
Xm = np.random.rand(m_size, n_size)  # Replace with actual dimensions
Ym = np.random.rand(m_size,n_size)  # Replace with actual dimensions

Zk, Lambda_k = schmid_DMD_method(Xm, Ym)

print("Ritz Vectors (Zk):")
print(Zk)

print("\nEigenvalues (Lambda_k):")
print(Lambda_k)

Ritz Vectors (Zk):
[[ 0.59932444+0.j         -0.41707879+0.1343754j  -0.41707879-0.1343754j ]
 [ 0.36969322+0.j         -0.43116111-0.20414205j -0.43116111+0.20414205j]
 [ 0.2304969 +0.j         -0.18657942+0.01554609j -0.18657942-0.01554609j]
 [ 0.32737444+0.j         -0.50575955-0.06572199j -0.50575955+0.06572199j]
 [ 0.58637385+0.j         -0.53327635+0.02941645j -0.53327635-0.02941645j]]

Eigenvalues (Lambda_k):
[[-0.31827169+0.j          0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.82053153+0.69764311j  0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.82053153-0.69764311j]]


DDMD_RRR algorithm

In [5]:
def ddmd_rrr(Xm, Ym, epsilon):
    # Step 2: Initialize Matrices
    Dx = np.diag(np.linalg.norm(Xm, axis=0)**2)
    X1m = Xm @ np.linalg.pinv(Dx)
    Y1m = Ym @ np.linalg.pinv(Dx)

    # Step 3: Thin SVD
    U, Sigma, Vh = np.linalg.svd(X1m, full_matrices=False)


    # Step 4: Determine Numerical Rank
    k = np.sum(Sigma > epsilon)


    # Step 5: Truncate SVD
    Uk = U[:, :k]
    Vk = Vh[:, :k]
    Sigma_k = np.diag(Sigma[:k])

    # Step 6: Data Driven Formula
    Bk = Y1m @ Vk @ np.linalg.inv(Sigma_k)

    # Step 7: Thin QR Factorization
    Q, R = np.linalg.qr(np.hstack((Uk, Bk)))


    diagonal_entries = np.diagonal(R)[:k]

    # Calculate the conjugate of each entry and store in a list
    diagonal_conjugates = [np.conjugate(entry) for entry in diagonal_entries]
    Sk = np.diag(diagonal_conjugates) @ R[:k, -k:]


    # Step 9: Compute Eigenvalues
    Lambda_k = np.linalg.eigvals(Sk)


    # Steps 10-13: Singular Value Decomposition Minimization, Construct Refined Ritz Vectors,
    # Optimal Residual, and Rayleigh Quotient
    Wk = np.zeros((k, k), dtype=complex)
    rk = np.zeros(k)
    rho_k = np.zeros(k, dtype=complex)

    for i in range(k):
        submatrix = R[:k, k + i:2 * k]
        lambda_i = Lambda_k[i]
        sigma_min_i, w_lambda_i = calculate_minimal_singular_value(R, k, lambda_i)
        Wk[:, i]  = w_lambda_i
        rk[i] = sigma_min_i
        rho_k[i] = np.vdot(Wk[:, i], Sk @ Wk[:, i])

    # Step 15: Output
    Zk = Uk @ Wk

    return Zk, Lambda_k, rk, rho_k




Example Usage of the method

In [6]:
# Replace Xm and Ym with your data, and adjust epsilon accordingly
Xm = np.random.rand(1000, 99)  # Replace with actual dimensions
Ym = np.random.rand(1000, 99)  # Replace with actual dimensions
epsilon = 1e-6  # Replace with the desired tolerance level

Zk, Lambda_k, rk, rho_k = ddmd_rrr(Xm, Ym, epsilon)

print("Refined Ritz Vectors (Zk):")
print(Zk)

print("\nEigenvalues (Lambda_k):")
print(Lambda_k)

print("\nOptimal Residuals (rk):")
print(rk)

print("\nRayleigh Quotients (rho_k):")
print(rho_k)

Refined Ritz Vectors (Zk):
[[-0.01099704+0.01082044j -0.01099704-0.01082044j -0.00990098+0.00090338j
  ... -0.00628596+0.00470483j -0.00628596-0.00470483j
   0.01171285+0.j        ]
 [ 0.01494247-0.03403957j  0.01494247+0.03403957j -0.04129495+0.00051441j
  ...  0.00278927+0.01981933j  0.00278927-0.01981933j
   0.01264765+0.j        ]
 [ 0.04126345+0.00992887j  0.04126345-0.00992887j -0.0144334 +0.00627293j
  ... -0.01569664-0.00152849j -0.01569664+0.00152849j
   0.01086691+0.j        ]
 ...
 [ 0.03418953-0.02010902j  0.03418953+0.02010902j  0.00249284-0.01689755j
  ...  0.00279525+0.00834174j  0.00279525-0.00834174j
   0.0074564 +0.j        ]
 [ 0.00330555+0.02033034j  0.00330555-0.02033034j -0.00077513+0.02623578j
  ...  0.00758616+0.05267538j  0.00758616-0.05267538j
  -0.05745268+0.j        ]
 [-0.05068783-0.00316005j -0.05068783+0.00316005j  0.01131449+0.01078395j
  ... -0.01180927-0.00592299j -0.01180927+0.00592299j
   0.00450135+0.j        ]]

Eigenvalues (Lambda_k):
[ 0.24626603